# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")
# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\ormis\Desktop\python-API-challenge\Output\Output.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,almyros,39.1822,22.7594,301.10,47,75,3.09,GR,1720031101
1,1,port-aux-francais,-49.3500,70.2167,273.42,63,98,2.50,TF,1720031102
2,2,puerto natales,-51.7236,-72.4875,275.40,75,100,2.06,CL,1720031104
3,3,san patricio,28.0170,-97.5169,307.68,58,53,6.60,US,1720031106
4,4,sake,-1.5736,29.0450,291.21,77,59,0.32,CD,1720031108


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
#%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lon",
    "Lat",
    geo=True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 550,
    color = "City",
    alpha = 0.7,
    s="Humidity",
    scale = 0.75
)

# Display the map
city_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[(city_data_df['Max Temp']<=290)\
                                  &(city_data_df['Max Temp']>=0)\
                                    &(city_data_df['Cloudiness']<=90)\
                                        &(city_data_df['Country']!='US')\
                                            &(city_data_df['Humidity']<80),:]
print(len(narrow_city_df))

# Drop any rows with null values
narrow_city_df.dropna()

# Display sample data
narrow_city_df.head()

61


,City_ID,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,hermanus,-34.4187,19.2345,285.82,78,51,8.25,ZA,1720031109
7,7,edinburgh of the seven seas,-37.0676,-12.3116,285.93,70,40,14.46,SH,1720031112
18,18,grytviken,-54.2811,-36.5092,269.19,61,4,1.83,GS,1720031127
40,40,penhalonga,-18.8911,32.6978,285.11,43,2,0.55,ZW,1720031160
42,42,nar'yan-mar,67.6713,53.0870,289.40,73,67,2.29,RU,1720031163


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lon,Humidity,Hotel Name
5,hermanus,ZA,-34.4187,19.2345,78,
7,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,
18,grytviken,GS,-54.2811,-36.5092,61,
40,penhalonga,ZW,-18.8911,32.6978,43,
42,nar'yan-mar,RU,67.6713,53.0870,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accomodation.hotel',\
          'filter':'',\
          'bias':'',\
          'apiKey':geoapify_key,\
          'limit':1
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lon']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'Circle: {lon},{lat},{radius}'
    params["bias"] = f'Proximity: {lon}, {lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hermanus - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
penhalonga - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
chonchi - nearest hotel: No hotel found
puerto deseado - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
springs - nearest hotel: No hotel found
aykhal - nearest hotel: No hotel found
yuzhno-kurilsk - nearest hotel: No hotel found
danielskuil - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
namtsy - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
berdigestyakh - nearest hotel: No hotel found
cill airne - nearest hotel: No hotel found
rovaniemi - nearest hotel: No hotel 

,City,Country,Lat,Lon,Humidity,Hotel Name
5,hermanus,ZA,-34.4187,19.2345,78,No hotel found
7,edinburgh of the seven seas,SH,-37.0676,-12.3116,70,No hotel found
18,grytviken,GS,-54.2811,-36.5092,61,No hotel found
40,penhalonga,ZW,-18.8911,32.6978,43,No hotel found
42,nar'yan-mar,RU,67.6713,53.0870,73,No hotel found
...,...,...,...,...,...,...
536,stony plain,CA,53.5334,-114.0021,77,No hotel found
540,akrehamn,NO,59.2667,5.1833,67,No hotel found
551,mar del plata,AR,-38.0023,-57.5575,70,No hotel found
561,vilyuysk,RU,63.7553,121.6247,72,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
#%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lon',
                    'Lat',
                    geo=True,
                    tiles = 'OSM',
                    frame_width = 750,
                    frame_height = 550,
                    color = 'City',
                    alpha = 0.75,
                    hover = True,
                    hover_cols = 'all',
                    use_index = False,
                    s='Humidity',
                    scale = 1.0)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lon,Lat]   (City,Humidity,Country,Hotel Name)